In [ ]:
import pandas as pd 
import numpy as np
import cv2
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity


In [ ]:
df = pd.read_csv("../pre-processing/cleaned_dataset.csv")  
df = df[["image_filename", "category", "style", "title", "price", "product_link"]]  

category_mapping = {cat: idx for idx, cat in enumerate(df["category"].unique())}
type_mapping = {typ: idx for idx, typ in enumerate(df["style"].unique())}

df["Category_Label"] = df["category"].map(category_mapping)
df["Type_Label"] = df["style"].map(type_mapping)

np.save("titles.npy", np.array(df["title"].values, dtype=object))
np.save("prices.npy", np.array(df["price"].values, dtype=object))
np.save("links.npy", np.array(df["link"].values, dtype=object))


print("✅ Titles, prices, and links saved successfully!")


✅ Titles, prices, and links saved successfully!


In [31]:
image_dir = "../pre-processing/processed_images/"

X, y_category, y_type = [], [], []  

for _, row in df.iterrows():
    img_path = os.path.join(image_dir, row["image_filename"])
    
    if os.path.exists(img_path):
        img = cv2.imread(img_path)
        img = cv2.resize(img, (224, 224))  
        img = img / 255.0  

        X.append(img)
        y_category.append(row["Category_Label"])
        y_type.append(row["Type_Label"])
    else:
        print(f"⚠️ Missing: {row['image_filename']}")

X = np.array(X, dtype=np.float32)
y_category = np.array(y_category, dtype=np.int32)
y_type = np.array(y_type, dtype=np.int32)

print(f"\n✅ Loaded {len(X)} images successfully!")


✅ Loaded 209 images successfully!


In [ ]:
X_train, X_val, y_train_category, y_val_category, y_train_type, y_val_type = train_test_split(
    X, y_category, y_type, test_size=0.2, random_state=42)

input_layer = keras.Input(shape=(224, 224, 3))

x = layers.Conv2D(32, (3, 3), activation='relu')(input_layer)
x = layers.MaxPooling2D((2, 2))(x)

x = layers.Conv2D(64, (3, 3), activation='relu')(x)
x = layers.MaxPooling2D((2, 2))(x)

x = layers.Conv2D(128, (3, 3), activation='relu')(x)
x = layers.MaxPooling2D((2, 2))(x)

x = layers.Flatten()(x)
x = layers.Dense(128, activation='relu')(x)
x = layers.Dropout(0.5)(x)

category_output = layers.Dense(len(category_mapping), activation='softmax', name="category_output")(x)
type_output = layers.Dense(len(type_mapping), activation='softmax', name="type_output")(x)

model = keras.Model(inputs=input_layer, outputs=[category_output, type_output])

model.compile(optimizer='adam',
              loss={'category_output': 'sparse_categorical_crossentropy', 
                    'type_output': 'sparse_categorical_crossentropy'},
              metrics={'category_output': ['accuracy'], 
                       'type_output': ['accuracy']})  


history = model.fit(X_train, 
                    {"category_output": y_train_category, "type_output": y_train_type}, 
                    epochs=10, 
                    validation_data=(X_val, {"category_output": y_val_category, "type_output": y_val_type}),
                    batch_size=32)

model.save("outfit_recommendation_model.h5")

print("✅ Model training completed and saved successfully!")


Epoch 1/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 4s 543ms/step - category_output_accuracy: 0.3146 - category_output_loss: 3.2473 - loss: 5.0505 - type_output_accuracy: 0.5115 - type_output_loss: 1.7068 - val_category_output_accuracy: 0.3333 - val_category_output_loss: 1.2606 - val_loss: 1.9879 - val_type_output_accuracy: 0.5000 - val_type_output_loss: 0.7030
Epoch 2/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 466ms/step - category_output_accuracy: 0.4448 - category_output_loss: 1.2364 - loss: 1.9064 - type_output_accuracy: 0.5827 - type_output_loss: 0.6657 - val_category_output_accuracy: 0.4762 - val_category_output_loss: 1.1408 - val_loss: 1.7721 - val_type_output_accuracy: 0.7619 - val_type_output_loss: 0.6365
Epoch 3/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 476ms/step - category_output_accuracy: 0.5873 - category_output_loss: 1.0373 - loss: 1.6807 - type_output_accuracy: 0.6166 - type_output_loss: 0.6439 - val_category_output_accuracy: 0.6667 - val_category_output_loss: 0.7645 - val_loss: 1.3927 - val_type_output_accu

✅ Model training completed and saved successfully!


In [ ]:
model = tf.keras.models.load_model("outfit_recommendation_model.h5")

feature_extractor_model = tf.keras.Model(inputs=model.input, outputs=model.layers[-3].output)
feature_extractor_model.build(input_shape=(None, 224, 224, 3))

print("✅ Feature extractor model is ready!")

dataset_folder = "../pre-processing/processed_images"
image_paths = []
dataset_features = []
dataset_categories = []
dataset_types = []

for _, row in df.iterrows():
    img_path = os.path.join(dataset_folder, row["image_filename"])
    
    if os.path.exists(img_path):
        img = cv2.imread(img_path)
        img = cv2.resize(img, (224, 224))
        img = img.astype(np.float32) / 255.0  
        img = np.expand_dims(img, axis=0)

        features = feature_extractor_model.predict(img)[0]
        dataset_features.append(features)
        image_paths.append(img_path)
        dataset_categories.append(row["Category_Label"])
        dataset_types.append(row["Type_Label"])

dataset_features = np.array(dataset_features)
np.save("dataset_features.npy", dataset_features)
np.save("image_paths.npy", np.array(image_paths))
np.save("dataset_categories.npy", np.array(dataset_categories))
np.save("dataset_types.npy", np.array(dataset_types))

print("✅ Dataset features & metadata saved successfully!")


✅ Feature extractor model is ready!
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━

In [ ]:
model = tf.keras.models.load_model("outfit_recommendation_model.h5")
feature_extractor_model = tf.keras.Model(inputs=model.input, outputs=model.layers[-3].output)

dataset_features = np.load("dataset_features.npy")
image_paths = np.load("image_paths.npy")
dataset_categories = np.load("dataset_categories.npy")
dataset_types = np.load("dataset_types.npy")

titles = np.load("titles.npy", allow_pickle=True)
prices = np.load("prices.npy", allow_pickle=True)
links = np.load("links.npy", allow_pickle=True)

def preprocess_image(image_path):
    img = cv2.imread(image_path)
    img = cv2.resize(img, (224, 224))
    img = img.astype(np.float32) / 255.0  
    img = np.expand_dims(img, axis=0) 
    return img

def recommend_outfit(query_image_path, top_n=8):
    query_img = preprocess_image(query_image_path)
    
    query_features = feature_extractor_model.predict(query_img)[0]
    query_category, query_type = model.predict(query_img)
    query_category = np.argmax(query_category)
    query_type = np.argmax(query_type)

    similarities = cosine_similarity([query_features], dataset_features)[0]

    valid_indices = [i for i in range(len(dataset_features)) if dataset_categories[i] == query_category and dataset_types[i] == query_type]

    filtered_similarities = [(i, similarities[i]) for i in valid_indices]
    filtered_similarities = sorted(filtered_similarities, key=lambda x: x[1], reverse=True)[:top_n]

    recommendations = [{
        "image": image_paths[i],
        "title": titles[i],
        "price": prices[i],
        "link": links[i],
        "similarity": round(score, 2)
    } for i, score in filtered_similarities]

    return recommendations

query_img_path = "y.jpg"
recommendations = recommend_outfit(query_img_path)

print("🎯 Recommended outfits:")
for i, rec in enumerate(recommendations):
    print(f"{i+1}. {rec['title']} - ₹{rec['price']}")
    print(f"   🖼️ Image: {rec['image']}")
    print(f"   🔗 Link: {rec['link']}")
    print(f"   🔍 Similarity Score: {rec['similarity']}")
    print("--------------------------------------------------")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
🎯 Recommended outfits:
1. Dokotoo Womens 2025 Formal Dresses Wrap V-Neck Ruched Sexy Bridesmaid Wedding Guest Maxi Dresses - ₹54
   🖼️ Image: ../pre-processing/processed_images/product_1740574906_26.jpg
   🔗 Link: https://www.amazon.com/sspa/click?ie=UTF8&spc=MTo0MzY3NjI3MDQ5Mzk3ODE1OjE3NDA1NzQ3Nzg6c3BfbXRmOjMwMDYyMzEzNTIwMDEwMjo6MDo6&url=%2FDokotoo-Elegant-Backless-Pleated-Bridesmaid%2Fdp%2FB0BBCNKQRJ%2Fref%3Dsr_1_25_sspa%3Fdib%3DeyJ2IjoiMSJ9.WQLmsddR0WRhuQAiBhs33e5C0tFhAnoAqxI9LqcbwkJWjVgxbvoKGvWR5yyXYTFAwNGQbJJK3gESGH2hrY5TjPtnW-XCAbPyreCIz0Yt8WwSAam0VBL8uqzlOypGFn0IzmFkxbMK3y5p211Vu_SXCSgSL1AsjsyNO3wVALfU69STN--NKWMk96r1Ea1vmfZ85ixejEdq4Yx0i8kv7Cvk8fhIC8pvUC50pfRyrhPOlEW3VSejQtqpdo1iBDg_7ta6wtzH1NgoRDrMznIW3GC6vHLVqga2b4oJqOY_UJiHbNI.YV8DNyXzfeQyl1fRQwsnUeENmIqJlbSFbha7nLyNLxI%26dib_tag%3Dse%26keywords%3Dformal%2Bdress%2Bfor%2Bwomen%26qid%3D1740574778%26sr%3D8-25-spons%26sp_csd%3Dd2lkZ2V0TmFtZT1zcF9tdGY%26p